In [12]:
#Imports
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import folium
import yaml

from eodag import EODataAccessGateway
from eodag import setup_logging
from eodag.crunch import FilterProperty
from eodag import SearchResult

from dotenv import dotenv_values

# Setup Verbose Values:
# 0: no logging and no progress bar
# 1: no logging but progress bars displayed
# 2: log at the INFO level
# 3: log at the DEBUG level (even more information)

setup_logging(verbose=0)

# EODAG - Search

EODAG (Earth Observation Data Access Gateway) is a command line tool and a Python package for searching and downloading remotely sensed images while offering a unified API for data access regardless of the data provider.

EODAG gives you an easy way to access products from more than 10 providers, with more than 50 different product types (Sentinel 1, Sentinel 2, Sentinel 3, Landsat, etc.) that can be searched and downloaded.

In [35]:
# Define all paths in paths.yml file. 
with open("paths.yml", 'r') as f:
    paths = yaml.safe_load(f)

In [36]:
#Get Secrets from .env File
secrets = dotenv_values(paths['credentials'])

In [38]:
#Create Folders for saving Data, serializing and post processing.

workspaces = [paths['download'], paths['serialize'], paths['post'], paths['shapefiles']]

for ws in workspaces:    
    if not os.path.isdir(ws):
        ws = os.path.abspath(ws)
        os.mkdir(ws)
        print(f'Created Folder: {ws}')
    else:
        ws = os.path.abspath(ws)
        print(f'Folder already exists: {ws}')

Folder already exists: c:\Users\npikall\eodag\eodag-workflows\data\eodag-downloads
Folder already exists: c:\Users\npikall\eodag\eodag-workflows\data\eodag-serialize
Folder already exists: c:\Users\npikall\eodag\eodag-workflows\data\eodag-postprocess
Folder already exists: c:\Users\npikall\eodag\eodag-workflows\data\shapefiles


## Step 1
### Configuration
In the configuration we pass the username and password from the Copernicus Dataspace Ecosystem (CDSE) to eodag. Also we define the path for the downloads.

In [ ]:
# 1. Configure
#Create EODAG Object and set preferred Provider

dag = EODataAccessGateway()
dag.set_preferred_provider("cop_dataspace") # Copernicus Data Space Ecosystem

dag.update_providers_config(f"""
    cop_dataspace:
        download:
            outputs_prefix: {os.path.abspath(paths['download'])}
        auth:
            credentials:
                username: {secrets['USER_KEY']}
                password: {secrets['USER_SECRET']}
""")

## Step 2
### Search
Searching for products is one of the most important features of eodag. Here you pass the desired parameters of your search.

In [ ]:
# 2. Search 
#Set Search Parameters and get catalog of products

latmin, latmax = 47.86, 48.407
lonmin, lonmax = 16.0, 16.9
extent = {'lonmin': lonmin, 'latmin': latmin, 'lonmax': lonmax, 'latmax': latmax}


search_results, total_count = dag.search(
    productType='S2_MSI_L2A',
    provider='cop_dataspace',
    geom=extent,
    #tileIdentifier="33UXP",
    start='2021-04-01',
    end='2021-05-01',
    cloudCover=50 #You can specify cloudcover here or later when filtering.
)

print(f"Got {len(search_results)} products and an estimated total number of {total_count} products.")

In [ ]:
# Folium Map
fmap = folium.Map(location=(np.array([latmin, latmax]).mean(), np.array([lonmin, lonmax]).mean()), zoom_start=9)
folium.Rectangle(bounds=[[latmin, lonmin],[latmax, lonmax]], color="red").add_to(fmap)
folium.GeoJson(
    data=search_results[:],  # SearchResult has a __geo_interface__ interface used by folium to get its GeoJSON representation, single results dont work (this [2:3] instead of [2])
    tooltip=folium.GeoJsonTooltip(fields=["title"])
).add_to(fmap)
fmap

In [ ]:
#Plot Quicklooks of Search Results
def plot_quicklooks(products):
    fig = plt.figure(figsize=(10,8))
    for i, product in enumerate(products[:12]):
        # This line takes care of downloading the quicklook
        quicklook_path = product.get_quicklook()
        
        date = product.properties['startTimeFromAscendingNode'][:16]
        provider = product.provider
        tile = product.properties['title'].split('_')[5].lstrip('T')
    
        # Plot the quicklook
        img = mpimg.imread(quicklook_path)
        ax = fig.add_subplot(3, 4, i+1)
        ax.set_title(f'Product {i}\n{date}\n{provider} - {tile}')
        ax.tick_params(top=False, bottom=False, left=False, right=False,
                       labelleft=False, labelbottom=False)
        plt.imshow(img)
    plt.tight_layout()
    
plot_quicklooks(search_results)

## Step 3
### Crunch
Crunching as defined in eodag is a way to filter the EO products contained in a SearchResult object. Several filters are available and further described in this document.

A SearchResult has a crunch() method that requires a filter instance as an argument, itself initialized with a dictionary that contains the required parameters. According to the filter used, some more kwargs may need to be passed to crunch(). The filters return a list of EOProducts.

The following Filter Options are available (Note `sr` is the search result):
- Filter by start and end date (`sr.filter_date()` or import `FilterDate` from eodag.crunch)
- Filter by geometry (`FilterOverlap`, )
- Filter by property (`FilterProperty`)
- Filter for online products (`sr.filter_online()` or `FilterProperty`)
- Filter the latest products intersecting a geometry (`sr.filter_latest_intersect()` or `FilterLatestIntersect`)




In [ ]:
# 3. Crunch (other Crunching Methods are described on EODAG Website)

#Filter for Cloudcover
filtered_products = search_results.crunch(
    FilterProperty(dict(cloudCover=100, operator="lt")))
print(f"{len(search_results) - len(filtered_products)} from {len(search_results)} products were filtered out by the property filter.")

#Filter already filterd products for online products
online_products = filtered_products.filter_online()
print(f"{len(filtered_products) - len(online_products)} from {len(online_products)} products are offline and have been filtered out.")

plot_quicklooks(online_products)



### Further Crunching

In order to have some more speciffic filtering operations at hand we can use the Nameing Convention, which stores a lot of information about the individual products. This enables us to filter for things like the relative orbits, the Tile Numbers or the processing baseline. 

#### Compact Naming Convention

The compact naming convention is arranged as follows:

``MMM_MSIXXX_YYYYMMDDHHMMSS_Nxxyy_ROOO_Txxxxx_< Product Discriminator >.SAFE``

The products contain two dates.

The first date (YYYYMMDDHHMMSS) is the datatake sensing time.
The second date is the "< Product Discriminator >" field, which is 15 characters in length, and is used to distinguish between different end user products from the same datatake. Depending on the instance, the time in this field can be earlier or slightly later than the datatake sensing time.
The other components of the filename are:

- ``MMM``: is the mission ID(S2A/S2B)
- ``MSIXXX``: MSIL1C denotes the Level-1C product level/ MSIL2A denotes the Level-2A product level
- ``YYYYMMDDHHMMSS``: the datatake sensing start time
- ``Nxxyy``: the Processing Baseline number (e.g. N0204)
- ``ROOO``: Relative Orbit number (R001 - R143)
- ``Txxxxx``: Tile Number field
- ``SAFE``: Product Format (Standard Archive Format for Europe)

The `EODAG` Search Results therefore can be filtered (maybe in a slightly unconventional way) by many more features as originally intedned by `EODAG`. Essentially you can use any property a product has (`product.properties`) to make an filtering operation. Here are some examples.

In [ ]:
# Sorting the search results for reproducibility (sorted by tile(5) and date (2))
products = SearchResult(sorted([p for p in online_products], key=lambda p: p.properties["title"].split("_")[5] + p.properties["title"].split("_")[2]))

# Filter by Processing Pipeline
products = SearchResult([p for p in products if p.properties['id'].split('_')[3] == 'N0300'])

# Filter by Tile
products = SearchResult([p for p in products if p.properties['id'].split('_')[5] == 'T33UXP'])
plot_quicklooks(products)

## Step 4 
### Serialize
The EODataAccessGateway class provides methods to save a SearchResult object to a GeoJSON file or to load a GeoJSON file to a SearchResult. These methods come in handy to save the state of a search and restore it later.

In [ ]:
# 4. Serialize (Save the filtered products as a GeoJSON File 
#               to save the state of a search and restore it later)

output_file = os.path.join(workspace_serialize, "search_results3.geojson")

dag.serialize(
    online_products,
    filename=output_file
)